In [1]:
import torch
from SchedMaint_env import GASU
from GASU_optimizer import optimize_GASU

In [2]:
# Episode Length: T
T = 31
state_horizon = 30
action_horizon = 1
simulation_days = T + state_horizon - action_horizon       # Total simulation days = 60
env_id = 'GASU-v0'
env = GASU(env_id, config_path="gasu_config.json")

# Preprocessing
# Step 1: Rendering essential information from environment
compressors = env.compressor_info()       # Provides the infromation compressors and its initial state in dict format
external_purchase_price = env.get_external_purchase_price()    # Provides the dict of external purchase price
external_purchase_capacity = env.get_external_purchase_capacity()    # Provides the dict of external purchase capacity

# Step 2: Initializing the optimizer by feeding the essential information from environment
optimizer = optimize_GASU(state_horizon, action_horizon, compressors)
optimizer._update_external_purchase_price(external_purchase_price)         # __Update the external purchase price in optimizer__
optimizer._update_external_purchase_capacity(external_purchase_capacity)   # __Update the external purchase capacity in optimizer__

### Simulation for 31 days (with optimal action)

In [3]:
env.reset()
flatt_state = env._get_state('flatt')  # mode = 'flatt'
opt_episode_reward = 0
simulation_episode_reward = 0
terminated = torch.tensor(False)

while terminated.item() is False:

    optimizer.update_state(flatt_state)
    objective_value = optimizer.solve()
    opt_cost = optimizer.action_horizon_cost()
    opt_action = optimizer._fetch_optimal_action_for_action_horizon()
    # feed action to environment, # tensor output
    flatt_state_tensor, reward, terminated, truncated, info = env.step(opt_action)
    flatt_state = env._get_state('flatt')
    opt_episode_reward += opt_cost
    simulation_episode_reward += reward.item()
    
print(f"Optimal Episode Reward: {opt_episode_reward}")
print(f"Final Episode Reward: {simulation_episode_reward}")

Optimal Episode Reward: 1221.8517861219732
Final Episode Reward: -1221.3104362487793


### Stochastic Case

In [ ]:
import json, copy, os, uuid
from itertools import product
import numpy as np
import torch
import torch
from SchedMaint_env import GASU
from GASU_optimizer import optimize_GASU

def write_scenario_json(nominal_cfg, deltas):
    cfg = copy.deepcopy(nominal_cfg)
    for comp, d in zip(cfg["compressors"], deltas):
        comp["mttf"] = max(comp["mttf"] - d, 1)

    fname = f"gasu_tmp_{uuid.uuid4().hex}.json"
    with open(fname, "w") as f:
        json.dump(cfg, f, indent=4)
    return fname


def run_episode(env, optimizer):
    env.reset()
    flatt_state = env._get_state("flatt")
    terminated  = torch.tensor(False)
    ep_cost     = 0.0

    while not terminated.item():
        optimizer.update_state(flatt_state)
        optimizer.solve()
        ep_cost += optimizer.action_horizon_cost()

        act = optimizer._fetch_optimal_action_for_action_horizon()
        _, _, terminated, _, _ = env.step(act)
        flatt_state = env._get_state("flatt")

    return ep_cost


if __name__ == "__main__":

    # ---------------- nominal config -----------------
    with open("gasu_config.json", "r") as f:
        nominal_cfg = json.load(f)

    T               = 31
    state_horizon   = 30
    action_horizon  = 1
    env_id          = "GASU-v0"

    scenario_costs = {}           # {(δ1,δ2,δ3): cost}

    for deltas in product([0, 1, 2], repeat=3):
        cfg_path = write_scenario_json(nominal_cfg, deltas)

        # ---------- env & optimiser for this scenario ----------
        env = GASU(env_id, config_path=cfg_path)
        compressors               = env.compressor_info()
        ext_price                 = env.get_external_purchase_price()
        ext_capacity              = env.get_external_purchase_capacity()

        optimizer = optimize_GASU(state_horizon, action_horizon, compressors)
        optimizer._update_external_purchase_price(ext_price)
        optimizer._update_external_purchase_capacity(ext_capacity)

        # ---------- roll out episode ----------
        episode_cost = run_episode(env, optimizer)
        scenario_costs[deltas] = episode_cost

        # optional: clean up the temporary file
        os.remove(cfg_path)

    avg_cost = np.mean(list(scenario_costs.values()))

    print("\n--- Episode cost per (δ1,δ2,δ3) ---")
    for k, v in scenario_costs.items():
        print(f"{k}: {v:.2f}")

    print(f"\nAverage optimal episode cost across 27 scenarios: {avg_cost:.2f}")

# average cost around 27 scenarios: 1226.96


--- Episode cost per (δ1,δ2,δ3) ---
(0, 0, 0): 1221.85
(0, 0, 1): 1221.86
(0, 0, 2): 1229.37
(0, 1, 0): 1221.23
(0, 1, 1): 1221.24
(0, 1, 2): 1228.75
(0, 2, 0): 1221.79
(0, 2, 1): 1221.80
(0, 2, 2): 1229.31
(1, 0, 0): 1224.63
(1, 0, 1): 1224.64
(1, 0, 2): 1224.65
(1, 1, 0): 1224.00
(1, 1, 1): 1224.01
(1, 1, 2): 1224.02
(1, 2, 0): 1224.56
(1, 2, 1): 1224.57
(1, 2, 2): 1224.58
(2, 0, 0): 1232.55
(2, 0, 1): 1232.56
(2, 0, 2): 1232.57
(2, 1, 0): 1231.92
(2, 1, 1): 1231.93
(2, 1, 2): 1231.94
(2, 2, 0): 1232.48
(2, 2, 1): 1232.49
(2, 2, 2): 1232.50

Average optimal episode cost across 27 scenarios: 1226.96
